In [ ]:
!pip install emcee

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.8 MB/s eta 0:00:00


In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from tqdm import tqdm

import numpy as np

In [28]:
class MLP(nn.Module):
    def __init__(self, layer_widths):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(28 * 28, layer_widths[0]))
        for i in range(len(layer_widths) - 1):
            self.layers.append(nn.Linear(layer_widths[i], layer_widths[i+1]))
        self.layers.append(nn.Linear(layer_widths[-1], 10))
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        for layer in self.layers[:-1]:
            x = self.relu(layer(x))
        x = self.layers[-1](x)  # No ReLU on the output layer
        return x

# Example usage
layer_widths = [30, 20, 10] # Example layer widths
model = MLP(layer_widths)

In [29]:
def train_mnist_model(model,
                      epochs=10,
                      batch_size=64,
                      learning_rate=0.001,
                      device='cuda' if torch.cuda.is_available() else 'cpu',
                      seed=123):
    """
    Train a PyTorch model on the MNIST dataset for classification.

    Parameters:
    - model: PyTorch model to be trained.
    - epochs: Number of epochs to train for.
    - batch_size: Batch size for the DataLoader.
    - learning_rate: Learning rate for the optimizer.
    - device: Device to train on ('cuda' or 'cpu').

    Returns:
    - model: Trained PyTorch model.
    """
    torch.manual_seed(seed)

    # Set the device
    model = model.to(device)

    # Define transformations for the MNIST dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),  # Normalize with mean and std of MNIST dataset
        lambda x: x.squeeze(0)
    ])

    # Load MNIST dataset
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Print epoch loss
        print(f"Epoch [{epoch+1}/{epochs}], Training Loss: {running_loss/len(train_loader):.4f}")

    ### Evaluation
    model.eval()

    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy_train = 100 * correct / total
    print(f"Accuracy on train set: {accuracy_train:.2f}%")

    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy_test = 100 * correct / total
    print(f"Accuracy on test set: {accuracy_test:.2f}%")

    return model, accuracy_train, accuracy_test

In [30]:
nodes = 10
min_layers = 2
max_layers = 10

layer_widths = [[nodes for _ in range(0, i)] for i in range(min_layers, max_layers+1)]

In [ ]:
epochs = 8
batch_size = 8
learning_rate = 0.001

data = []
for layer_width in layer_widths:
  print(f'Training model with widths: {layer_width}')
  model = MLP(layer_width)
  model_trained, accuracy_train, accuracy_test = train_mnist_model(model=model,
                                                                  epochs=epochs,
                                                                  batch_size=batch_size,
                                                                  learning_rate=learning_rate)
  data.append([layer_width, accuracy_train, accuracy_test])

100%|██████████| 7500/7500 [00:28<00:00, 263.27it/s]


Epoch [1/8], Loss: 0.3967


100%|██████████| 7500/7500 [00:31<00:00, 235.64it/s]


Epoch [2/8], Loss: 0.2865


100%|██████████| 7500/7500 [00:28<00:00, 262.12it/s]


...

100%|██████████| 7500/7500 [00:31<00:00, 239.18it/s]


Epoch [4/8], Loss: 0.2734


100%|██████████| 7500/7500 [00:30<00:00, 243.06it/s]


Epoch [5/8], Loss: 0.2584


 14%|█▍        | 1067/7500 [00:04<00:25, 248.76it/s]